In [7]:
!pip install gspread

In [8]:
import gspread
 
# define the scope
scopes = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']
client = gspread.service_account('./secret/questioniso-0207dbbf89c0.json', scopes=scopes)

In [9]:
# client = gspread.authorize(creds)
gsheet_id = '1j84wJRX7xoszkF0B_BFIe_XdNEuL0MvlnoHfI5ardgY'
spreadsheet = client.open_by_key(gsheet_id)
# sheet = client.open_by_key(gsheet_id).Sheet1

In [10]:
# List all sheets
sheets = spreadsheet.worksheets()

# Loop through each sheet
for sheet in sheets:
    print(f"Sheet title: {sheet.title}")
    
    # Get the values in the range F12:F50
    cell_range = sheet.range("F12:F50")
    
    # Loop through each cell in the range
    for cell in cell_range:
        print(f"Row {cell.row}, Column {cell.col}: {cell.value}")

Sheet title: History of changes
Row 12, Column 6: 
Row 13, Column 6: 
Row 14, Column 6: 
Row 15, Column 6: 
Row 16, Column 6: 
Row 17, Column 6: 
Row 18, Column 6: 
Row 19, Column 6: 
Row 20, Column 6: 
Row 21, Column 6: 
Row 22, Column 6: 
Row 23, Column 6: 
Row 24, Column 6: 
Row 25, Column 6: 
Row 26, Column 6: 
Row 27, Column 6: 
Row 28, Column 6: 
Row 29, Column 6: 
Row 30, Column 6: 
Row 31, Column 6: 
Row 32, Column 6: 
Row 33, Column 6: 
Row 34, Column 6: 
Row 35, Column 6: 
Row 36, Column 6: 
Row 37, Column 6: 
Row 38, Column 6: 
Row 39, Column 6: 
Row 40, Column 6: 
Row 41, Column 6: 
Row 42, Column 6: 
Row 43, Column 6: 
Row 44, Column 6: 
Row 45, Column 6: 
Row 46, Column 6: 
Row 47, Column 6: 
Row 48, Column 6: 
Row 49, Column 6: 
Row 50, Column 6: 
Sheet title: DBテーブル一覧
Row 12, Column 6: 
Row 13, Column 6: 
Row 14, Column 6: 
Row 15, Column 6: 
Row 16, Column 6: 
Row 17, Column 6: 
Row 18, Column 6: 
Row 19, Column 6: 
Row 20, Column 6: 
Row 21, Column 6: 
Row 22, Column 

In [113]:
import pandas as pd
sheets = spreadsheet.worksheets()
# print(sheets)

sheet_by_name = spreadsheet.worksheet('members')
print(sheet_by_name)

<Worksheet 'members' id:1847736165>


In [12]:
sheet_members = sheets[7]

In [13]:
df_members = pd.DataFrame(sheet_members.get_all_records(expected_headers=[]))

In [14]:
print(df_members.head())

                     Database Design                   No: BM-2-901-50
0                                                          Version: 01
1                                               Issue Date: 07/07/2021
2                                     ISO/IEC 27001:2013&ISO 9001:2015
3  作成日\nCreated date                                                  
4       作成者\nCreator                                                  


In [23]:
val = sheet_members.get('F12:F20')
print(val)

[['id'], ['name'], ['email'], ['authority_id'], ['created_at'], ['updated_at']]


In [125]:
# https://python.langchain.com/docs/how_to/query_multiple_queries/

from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_community.utilities import SQLDatabase
from typing import List, Optional
from pydantic import BaseModel, Field
from enum import Enum
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o")

In [126]:
class MySQLDataType(str, Enum):
    CHAR = "CHAR"
    VARCHAR = "VARCHAR"
    BINARY = "BINARY"
    VARBINARY = "VARBINARY"
    TINYBLOB = "TINYBLOB"
    BLOB = "BLOB"
    MEDIUMBLOB = "MEDIUMBLOB"
    LONGBLOB = "LONGBLOB"
    TINYTEXT = "TINYTEXT"
    TEXT = "TEXT"
    MEDIUMTEXT = "MEDIUMTEXT"
    LONGTEXT = "LONGTEXT"
    ENUM = "ENUM"
    SET = "SET"
    BIT = "BIT"
    TINYINT = "TINYINT"
    BOOL = "BOOL"
    BOOLEAN = "BOOLEAN"
    SMALLINT = "SMALLINT"
    MEDIUMINT = "MEDIUMINT"
    INT = "INT"
    INTEGER = "INTEGER"
    BIGINT = "BIGINT"
    FLOAT = "FLOAT"
    DOUBLE = "DOUBLE"
    DECIMAL = "DECIMAL"
    DATE = "DATE"
    DATETIME = "DATETIME"
    TIMESTAMP = "TIMESTAMP"
    TIME = "TIME"
    YEAR = "YEAR"
    GEOMETRY = "GEOMETRY"
    POINT = "POINT"
    LINESTRING = "LINESTRING"
    POLYGON = "POLYGON"
    MULTIPOINT = "MULTIPOINT"
    MULTILINESTRING = "MULTILINESTRING"
    MULTIPOLYGON = "MULTIPOLYGON"
    GEOMETRYCOLLECTION = "GEOMETRYCOLLECTION"

class ColumnType(BaseModel):
    """ most relavance MySQL data type for the given column """
    column_type: MySQLDataType = Field(
        description="most relavance MySQL data type",
    )
    column_name: str = Field(
        description="Column name",
    )

@tool
def suggest_data_type(value: str) -> ColumnType:
    """Suggests the most appropriate MySQL data type for the given column name."""
    system = "You are an expert in database design. Suggest the most appropriate MySQL data type for the given column name based on its name and typical usage in an ERD table structure."
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system),
            ("human", f"Column name: {value}")
        ]
    )
    structured_llm = llm.with_structured_output(ColumnType)
    chain = {"input": RunnablePassthrough()} | prompt | structured_llm

    prompt_value = chain.invoke({"input": value})
    return prompt_value

In [111]:
# for t in val:
#     print(t)
#     print(extract_data_type(t[0]))
#     # print(think(t[0]))
# # sheet_members.update_acell('B1', 'Bingo!')


['id']
column_type=<MySQLDataType.INT: 'INT'> column_name='id'
['name']
column_type=<MySQLDataType.VARCHAR: 'VARCHAR'> column_name='name'
['email']
column_type=<MySQLDataType.VARCHAR: 'VARCHAR'> column_name='email'
['authority_id']
column_type=<MySQLDataType.INT: 'INT'> column_name='authority_id'
['created_at']
column_type=<MySQLDataType.DATETIME: 'DATETIME'> column_name='created_at'
['updated_at']
column_type=<MySQLDataType.DATETIME: 'DATETIME'> column_name='updated_at'


In [119]:


@tool
def read_cell(sheet_name: str, cell: str) -> str:
    """Reads the value from the specified cell in A1 notation and sheet name."""
    return spreadsheet.worksheet(sheet_name).acell(cell).value

@tool
def write_cell(sheet_name: str, cell: str, value: str) -> None:
    """Writes the value to the specified cell in A1 notation and sheet name."""
    spreadsheet.worksheet(sheet_name).update_acell(cell, value)

@tool
def read_values(sheet_name: str, range: str) -> List[List[str]]:
    """Reads the values from the specified range in A1 notation and sheet name."""
    return spreadsheet.worksheet(sheet_name).get(range)

@tool
def write_values(sheet_name: str, range: str, values: List[List[str]]) -> None:
    """Writes the values to the specified range in A1 notation and sheet name."""
    spreadsheet.worksheet(sheet_name).update(range, values)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant specialized in interacting with Google Sheets. Respond to the user's queries and commands."),
        ("human", "{input}"),
        # Placeholders fill up a **list** of messages
        ("placeholder", "{agent_scratchpad}"),
    ]
)



tools = [read_cell, write_cell, read_values, write_values, suggest_data_type]

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(agent=agent, tools=tools)


In [120]:
query = "Get value from cell with range [F12:F20] in the sheet named 'members'"
agent_executor.invoke({"input": query})

{'input': "Get value from cell with range [F12:F20] in the sheet named 'members'",
 'output': "The values from the range F12:F20 in the 'members' sheet are as follows:\n\n- F12: id\n- F13: name\n- F14: email\n- F15: authority_id\n- F16: created_at\n- F17: updated_at"}

In [121]:
query = """
Get value from cell with range [F12:F20] in the sheet named 'members',
each value should be converted to the most appropriate MySQL data type and fill the result
in the G{same row index}.G{same row index} cell in the same sheet.
Ex: F12 value is 'user_name', the G12 cell should be filled with 'VARCHAR'.
"""

agent_executor.invoke({"input": query})

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_34454/3244738919.py:24: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  spreadsheet.worksheet(sheet_name).update(range, values)


{'input': "\nGet value from cell with range [F12:F20] in the sheet named 'members',\neach value should be converted to the most appropriate MySQL data type and fill the result\nin the G{same row index}.G{same row index} cell in the same sheet.\nEx: F12 value is 'user_name', the G12 cell should be filled with 'VARCHAR'.\n",
 'output': 'The values from cells F12 to F20 in the "members" sheet have been converted to the appropriate MySQL data types and filled in the corresponding G column cells as follows:\n\n- F12 (id) -> G12 (INT)\n- F13 (name) -> G13 (VARCHAR)\n- F14 (email) -> G14 (VARCHAR)\n- F15 (authority_id) -> G15 (INT)\n- F16 (created_at) -> G16 (DATETIME)\n- F17 (updated_at) -> G17 (TIMESTAMP)'}

In [122]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

# Define the graph

from langgraph.prebuilt import create_react_agent

graph = create_react_agent(llm, tools=tools, checkpointer=memory)

In [123]:
config = {"configurable": {"thread_id": "1"}}
inputs = {"messages": [("user", query)]}

graph.invoke(inputs, config=config)

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_34454/3244738919.py:24: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  spreadsheet.worksheet(sheet_name).update(range, values)


{'messages': [HumanMessage(content="\nGet value from cell with range [F12:F20] in the sheet named 'members',\neach value should be converted to the most appropriate MySQL data type and fill the result\nin the G{same row index}.G{same row index} cell in the same sheet.\nEx: F12 value is 'user_name', the G12 cell should be filled with 'VARCHAR'.\n", additional_kwargs={}, response_metadata={}, id='b982eb20-44d0-4ecd-9243-31cab24085a3'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ryw8E3I89Mp99PQGdmkMrcVX', 'function': {'arguments': '{"range":"F12:F20","sheet_name":"members"}', 'name': 'read_values'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 277, 'total_tokens': 300, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-

In [124]:
query_2 = "apply similar action for sheet named 'ship_labor_holidays'"

graph.invoke({"messages": [("user", query_2)]}, config=config)

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_34454/3244738919.py:24: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  spreadsheet.worksheet(sheet_name).update(range, values)


{'messages': [HumanMessage(content="\nGet value from cell with range [F12:F20] in the sheet named 'members',\neach value should be converted to the most appropriate MySQL data type and fill the result\nin the G{same row index}.G{same row index} cell in the same sheet.\nEx: F12 value is 'user_name', the G12 cell should be filled with 'VARCHAR'.\n", additional_kwargs={}, response_metadata={}, id='b982eb20-44d0-4ecd-9243-31cab24085a3'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ryw8E3I89Mp99PQGdmkMrcVX', 'function': {'arguments': '{"range":"F12:F20","sheet_name":"members"}', 'name': 'read_values'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 277, 'total_tokens': 300, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-

In [15]:
import pandas as pd
import lotus
from lotus.models import LM

# configure the LM, and remember to export your API key
lm = LM(model="gpt-4o-mini")
lotus.settings.configure(lm=lm)

# create dataframes with course names and skills
courses_data = {
    "Course Name": [
        "History of the Atlantic World",
        "Riemannian Geometry",
        "Operating Systems",
        "Food Science",
        "Compilers",
        "Intro to computer science",
    ]
}
skills_data = {"Skill": ["Math", "Computer Science"]}
courses_df = pd.DataFrame(courses_data)
skills_df = pd.DataFrame(skills_data)

# lotus sem join 
res = courses_df.sem_join(skills_df, "Taking {Course Name} will help me learn {Skill}")
print(res)

Join comparisons: 100%|██████████ 12/12 LM Calls [00:03<00:00,  3.44it/s]

                 Course Name             Skill
1        Riemannian Geometry              Math
2          Operating Systems  Computer Science
4                  Compilers              Math
4                  Compilers  Computer Science
5  Intro to computer science              Math
5  Intro to computer science  Computer Science
